<a href="https://colab.research.google.com/github/SurajitBiswasPhysics/surajit-biswas.github.io/blob/main/fcc_predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
dataset.info()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

label_encoder = LabelEncoder()

In [ ]:
# Encode 'sex', 'smoker', 'region' columns
dataset['sex'] = label_encoder.fit_transform(dataset['sex'])
dataset['smoker'] = label_encoder.fit_transform(dataset['smoker'])
dataset['region'] = label_encoder.fit_transform(dataset['region'])

In [ ]:
# Step 4: Split data into features (X) and labels (y)
X = dataset.drop(columns=['expenses'])  # Features
y = dataset['expenses']  # Label

In [ ]:
# Step 5: Split into training and testing datasets (80% train, 20% test)
train_dataset, test_dataset, train_labels, test_labels = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Step 6: Train the regression model
model = LinearRegression()
model.fit(train_dataset, train_labels)

In [ ]:
# Step 7: Evaluate the model
predictions = model.predict(test_dataset)
mae = mean_absolute_error(test_labels, predictions)
print(f"Mean Absolute Error: {mae}")

In [ ]:
# Check if the MAE is under 3500
if mae < 3500:
    print("Model meets the target MAE!")
else:
    print("Model does not meet the target MAE.")

In [ ]:
# Step 8: Plot the results
plt.scatter(test_labels, predictions)
plt.xlabel('True Expenses')
plt.ylabel('Predicted Expenses')
plt.title('True vs Predicted Healthcare Expenses')
plt.show()

In [ ]:
# Check correlation
import seaborn as sns

correlation = dataset.corr()
sns.heatmap(correlation, annot=True, cmap="coolwarm")
plt.show()

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
